## Load dependencies

In [1]:
import pandas as pd
import time
import os
import sys
path = os.path.abspath('../')
print(f"The project path is {path}")
sys.path.append(path)
from demand.entities.pointofinterest import PointOfInterestCollection
from demand.handlers.populationdatahandler import PopulationDataHandler
from demand.demand.demand import Demand
from demand.utils import setup_logger, generate_dataset_id

The project path is c:\Users\peytrign\Documents\demand-estimator


## Data config
Edit the below cell to point to the correct data
* `country_code` of the desired data
* pointofinterest is the name of the file containing the pointofinterest , it is located in `/data/country_code/input`

Example:
```python 
country_code = 'ESP'
dataset_ids = dict(
    pointofinterest = 'ESP-1697915895-xs2u-pointofinterest.csv',
)

```

In [2]:
country_code = 'ESP'
dataset_id = 'ESP-1697915895-xs2u-pointofinterest.csv'

## Analysis config

In [3]:
radii = [1, 2, 3]
radius_for_demand = 1
dataset_year = 2020
one_km_res = True
un_adjusted = True
overlap_allowed = False
mbps_demand_per_user = 5
are_poi_schools = False

## Helpers

In [4]:
# Set up a logger
logger = setup_logger(f'pcd_{country_code}', log_filepath=f'{path}/logs/{country_code}-{round(time.time())}-pcd.log')

In [5]:
# Set up the population data handler
population_data_handler = PopulationDataHandler(
    data_dir = f'{path}/data/population',
    country_code = country_code,
    dataset_year = dataset_year,
    one_km_res = one_km_res,
    un_adjusted = un_adjusted,
    logger = logger)

## Load data

In [6]:
# Load point of interest data as records
filepath = f"{path}/data/{country_code}/input/{dataset_id}"
entity_records = pd.read_csv(filepath).to_dict('records')

# Load population data into a collection
pointofinterest_collection = PointOfInterestCollection()
pointofinterest_collection.load_from_records(entity_records)

## Initialize `Demand` class

In [7]:
demand = Demand(
    points_of_interest = pointofinterest_collection,
    population_data_handler = population_data_handler,
    radii = radii,
    logger = logger,
    overlap_allowed = overlap_allowed,
    mbps_demand_per_user = mbps_demand_per_user,
    are_poi_schools = are_poi_schools)

2024-11-25 14:29:24,378 - pcd_ESP - INFO - Processing population tif file...
2024-11-25 14:29:24,639 - pcd_ESP - INFO - Population tif file is processed!


## Perform Analysis

In [8]:
demand.perform_analysis()

2024-11-25 14:29:25,681 - pcd_ESP - INFO - Creating Voronoi diagram around POIs...
2024-11-25 14:29:25,783 - pcd_ESP - INFO - Overlaying population data for 1km radius around a point of interest...
2024-11-25 14:29:26,635 - pcd_ESP - INFO - Gathering point of interest count for 1km radius around point...
2024-11-25 14:29:26,743 - pcd_ESP - INFO - Overlaying population data for 2km radius around a point of interest...
2024-11-25 14:29:27,266 - pcd_ESP - INFO - Gathering point of interest count for 2km radius around point...
2024-11-25 14:29:27,305 - pcd_ESP - INFO - Overlaying population data for 3km radius around a point of interest...
2024-11-25 14:29:27,883 - pcd_ESP - INFO - Gathering point of interest count for 3km radius around point...


Demand Analysis Summary:
Number of points of interest: 100
Country population count: 46754784.0
Mean population count of 1km area around a point of interest: 742.58
Median population count of 1km area around a point of interest: 157.5
Sum population count of 1km area around a point of interest: 74258
Mean point of interest count of 1km area around a point of interest: 1.0
Median point of interest count of 1km area around a point of interest: 1.0
Sum point of interest count of 1km area around a point of interest: 100
Mean population count of 2km area around a point of interest: 1418.36
Median population count of 2km area around a point of interest: 366.0
Sum population count of 2km area around a point of interest: 141836
Mean point of interest count of 2km area around a point of interest: 1.0
Median point of interest count of 2km area around a point of interest: 1.0
Sum point of interest count of 2km area around a point of interest: 100
Mean population count of 3km area around a point o

In [9]:
d_results = demand.get_results_table()

In [10]:
d_results.head()

,poi_id,population,poi_count,number_of_users,total_mbps
0,23dd6a45-3656-435b-b3b1-c16efab9daeb,"{'population_1km': 328, 'population_2km': 1097...","{'poi_count_1km': 1, 'poi_count_2km': 1, 'poi_...",328,1640
1,e13a657c-edf0-4013-92db-6c70136e3ac9,"{'population_1km': 24, 'population_2km': 32, '...","{'poi_count_1km': 1, 'poi_count_2km': 1, 'poi_...",24,120
2,de75c87b-2676-47be-8454-4c44c4e6f644,"{'population_1km': 103, 'population_2km': 207,...","{'poi_count_1km': 1, 'poi_count_2km': 1, 'poi_...",103,515
3,4267fc81-0e9f-40ca-84c8-84529958dc22,"{'population_1km': 149, 'population_2km': 326,...","{'poi_count_1km': 1, 'poi_count_2km': 1, 'poi_...",149,745
4,ae0ccc6e-5f91-4a58-a60b-68e8ebcdc797,"{'population_1km': 213, 'population_2km': 213,...","{'poi_count_1km': 1, 'poi_count_2km': 1, 'poi_...",213,1065


## Save Results

In [12]:
dataset_id = generate_dataset_id(prefix=country_code)
output_folder = f'{path}/data/{country_code}/output/'
os.makedirs(output_folder, exist_ok=True)
d_results.to_json(f'{output_folder}/{dataset_id}-demand.json')